In [9]:
import cv2
import numpy as np
import os

# Set up the directory paths for multiple images
image_folder = r'C:\Users\admin\Desktop\Pytorch_ObjectDetection\Images'  # Update this path to your folder containing images
output_folder = r'C:\Users\admin\Desktop\Pytorch_ObjectDetection\output'  # Update this path to where the output images will be saved
config_path = "yolov3.cfg"
weights_path = "yolov3.weights"
classes_path = "coco.names"

# Function to load YOLO output layers
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# Function to draw bounding boxes and labels on the image
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id % len(COLORS)]
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Load class names from file
with open(classes_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Generate random colors for each class
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Load YOLO model
net = cv2.dnn.readNet(weights_path, config_path)

# Iterate through all images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Check for image file extensions
        image_path = os.path.join(image_folder, filename)
        
        # Load the image
        image = cv2.imread(image_path)

        Width = image.shape[1]
        Height = image.shape[0]
        scale = 0.00392

        # Prepare the input image for the network
        blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)
        net.setInput(blob)

        # Get output layers and run forward pass
        outs = net.forward(get_output_layers(net))

        # Initialize lists for detected objects
        class_ids = []
        confidences = []
        boxes = []
        conf_threshold = 0.3  # Adjust the confidence threshold as per your requirements
        nms_threshold = 0.4  # Adjust NMS to eliminate too many overlapping boxes

        # Process network outputs
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > conf_threshold:
                    center_x = int(detection[0] * Width)
                    center_y = int(detection[1] * Height)
                    w = int(detection[2] * Width)
                    h = int(detection[3] * Height)
                    x = center_x - w / 2
                    y = center_y - h / 2
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    boxes.append([x, y, w, h])

        # Perform non-maximum suppression to eliminate redundant boxes
        indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

        # Draw detected objects
        for i in indices:
            try:
                box = boxes[i]
            except:
                i = i[0]
                box = boxes[i]
            
            x = box[0]
            y = box[1]
            w = box[2]
            h = box[3]
            draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x + w), round(y + h))

        # Save the output image with bounding boxes
        output_image_path = os.path.join(output_folder, f"detected_{filename}")
        cv2.imwrite(output_image_path, image)
        print(f"Processed {filename} and saved to {output_image_path}")

print("Processing complete.")


Processed baggage_claim.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_baggage_claim.jpg
Processed dining_table.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_dining_table.jpg
Processed Hall.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_Hall.jpg
Processed hall_view.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_hall_view.jpg
Processed home-interior-design-ideas.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_home-interior-design-ideas.jpg
Processed living_room.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_living_room.jpg
Processed office-interior-designing-service.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_office-interior-designing-service.jpg
Processed Office_view.jpg and saved to C:\Users\admin\Desktop\Pytorch_ObjectDetection\output\detected_Office_vie